In [33]:
# Import necessary packages
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [34]:
import os
os.chdir('j:\Studio De Voogt\Specialists & NA\Feadship Database\Feadship Python\JOSHA_workingstudent_2024\Main Dimensions Depth')
current_path = os.getcwd()
print(current_path)

j:\Studio De Voogt\Specialists & NA\Feadship Database\Feadship Python\JOSHA_workingstudent_2024


In [35]:
# Prepare dataset
tabel = pd.read_excel('Main Dimensions Cleaned.xlsx')
tabel = tabel.dropna(subset=['Length Over All', 'Draught Design Deadweight', 'Depth', 'Beam Moulded'])  # Remove missing values

print(tabel)

            Number                       Yard  Length Over All  \
0           DN3423                        NaN           128.80   
1     YN831 DN3365  Royal van Lent, Amsterdam           143.50   
2           DN3331                        NaN           136.00   
3          DN2631                         NaN           180.00   
4           DN3403                        NaN           131.00   
...            ...                        ...              ...   
1020       DN1653                         NaN            25.30   
1021       DN1611                         NaN             9.25   
1022       DN1589                         NaN            11.45   
1023       DN1602                         NaN             7.00   
1024        dn3590                        NaN            95.00   

      Draught Design Deadweight  Depth  Beam Moulded  
0                          6.00  11.55         22.50  
1                          5.95  11.35         20.00  
2                          5.83   9.93    

In [36]:
# Rename variables
Loa = tabel['Length Over All']
Tfull = tabel['Draught Design Deadweight']
D = tabel['Depth']
Bmld = tabel['Beam Moulded']

In [37]:
# Define variables for regression
x = tabel.loc[:, ~tabel.columns.isin(['Number', 'Yard', 'Depth'])]
y = D

# Split into training and testing sets (80-20 split)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42) #25% test set and 75% training set

# Add constant to training set for intercept
x_train = sm.add_constant(x_train)
x_test = sm.add_constant(x_test)

# Run regression
model = sm.OLS(y_train, x_train)
results = model.fit(cov_type="HC0")
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  Depth   R-squared:                       0.963
Model:                            OLS   Adj. R-squared:                  0.962
Method:                 Least Squares   F-statistic:                     3763.
Date:                Tue, 29 Oct 2024   Prob (F-statistic):               0.00
Time:                        15:55:50   Log-Likelihood:                -160.85
No. Observations:                 717   AIC:                             329.7
Df Residuals:                     713   BIC:                             348.0
Df Model:                           3                                         
Covariance Type:                  HC0                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [38]:
# Calculate predicted values
predicted_values = results.predict(x_test)

# Check residuals for normality
residuals_OLS = y_test - predicted_values

# Add column to dataframe
tabel.loc[x_test.index, 'Predicted_Depth_OLS'] = predicted_values
test_results = tabel.loc[x_test.index, ['Depth', 'Predicted_Depth_OLS']]

print(test_results.head())

     Depth  Predicted_Depth_OLS
527    5.1             5.521533
359    6.0             6.076129
447    4.8             5.376403
31     8.0             7.595722
621    4.5             4.587326


In [39]:
# Calculate MSE
mse = mean_squared_error(y_test, predicted_values)
print(f"Mean Squared Error (MSE): {mse}")

# Bereken RMSE
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Mean Squared Error (MSE): 0.09574951099783663
Root Mean Squared Error (RMSE): 0.30943417878094304


### Predict Depth: fill in your specific values here! 

In [40]:
# Fill in the green numbers which correspond to the variable names in red before it:

Depth_prediction = results.params['const'] +(results.params['Length Over All']*95.65) +(results.params['Beam Moulded']*14)+(results.params['Draught Design Deadweight']*3.85)
print('Dit is de voorspelling volgens robuuste OLS:', Depth_prediction)

Dit is de voorspelling volgens robuuste OLS: 7.208595344757467
